In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#read log generated from MIMIC-IV Event Log Generation Tool
transfers = pd.read_csv("EL_2.csv")

In [ ]:
transfers = transfers.rename(columns={"case:hadm_id":"hadm_id", "time:timestamp":"intime", "concept:name":"careunit"})

In [ ]:
transfers = transfers.sort_values(["hadm_id", "intime"])

In [ ]:
transfers = transfers.loc[~transfers["hadm_id"].isna()]

In [ ]:
transfers.reset_index(inplace=True)

In [ ]:
transfers.drop("index", axis=1, inplace=True)

In [ ]:
hf_t = transfers

In [ ]:
hf_t["intime"] = pd.to_datetime(hf_t["intime"])
hf_t["outtime"] = pd.to_datetime(hf_t["outtime"])

In [ ]:
hf_t.loc[hf_t["outtime"].isna(), "outtime"] = hf_t["intime"] + pd.Timedelta(seconds=1)

In [ ]:
hf_t.reset_index(inplace=True)
hf_t.drop("index", axis=1, inplace=True)

In [ ]:
hf_t.loc[(hf_t["careunit"].isna()) & (hf_t["eventtype"] == "discharge"), "careunit"] = "Discharged"
hf_t.loc[hf_t["eventtype"] == "discharge", "careunit"] = "Discharged"

In [ ]:
hf_t.loc[(hf_t["careunit"].isna()), "careunit"] = "Unknown"

In [ ]:
hf_t

In [ ]:
icu = list(hf_t.loc[hf_t["careunit"].str.contains("Care Unit")]["hadm_id"])

In [ ]:
first_dep = hf_t.groupby("hadm_id").first()

In [ ]:
hadms_f_d = list(first_dep.loc[first_dep["careunit"] == "Emergency Department"].reset_index()["hadm_id"])

In [ ]:
hf_t_ed = hf_t.loc[hf_t["hadm_id"].isin(hadms_f_d)]

In [ ]:
hf_t_ed = hf_t_ed.loc[hf_t["hadm_id"].isin(icu)]

In [ ]:
hf_icu = hf_t_ed

In [ ]:
hf_icu.loc[(hf_icu["careunit"]=="Emergency Department"), "department"] = "Emergency Department"

In [ ]:
hf_icu.loc[(hf_icu["careunit"].str.contains("CCU")) | (hf_icu["careunit"].str.contains("CVICU")), "department"] = "Cardiac ICU"

In [ ]:
hf_icu.loc[(hf_icu["careunit"].str.contains("SICU")), "department"] = "Surgical ICU"

In [ ]:
hf_icu.loc[(hf_icu["careunit"].str.contains("MICU")) & (~hf_icu["careunit"].str.contains("SICU")), "department"] = "Medical ICU"

In [ ]:
hf_icu.loc[(hf_icu["careunit"]== "Medicine"), "department"] = "Medicine"

In [ ]:
hf_icu.loc[(hf_icu["careunit"]=="Medicine/Cardiology") | (hf_icu["careunit"]=="Medicine/Cardiology Intermediate"), "department"] = "Cardiology"

In [ ]:
hf_icu.loc[(hf_icu["careunit"]=="Cardiology Surgery Intermediate") | (hf_icu["careunit"]=="Cardiology Surger") | (hf_icu["careunit"].str.contains("Med/Surg")), "department"] = "Surgery"

In [ ]:
hf_icu.loc[(hf_icu["careunit"]== "Discharged"), "department"] = "Discharged"

In [ ]:
to_remove = list(hf_icu.loc[hf_icu["department"].isna()]["hadm_id"].unique())

In [ ]:
hf_icu_clean = hf_icu.loc[~hf_icu["hadm_id"].isin(to_remove)]

In [ ]:
hf_icu_clean["transfer_duration"] = hf_icu_clean["outtime"] - hf_icu_clean["intime"]

In [ ]:
hf_icu_clean = hf_icu_clean.reset_index().sort_values(["hadm_id", "intime"]).drop("index", axis=1)

In [ ]:
hf_icu_clean

In [ ]:
newdf = pd.DataFrame(data=None, columns=hf_icu_clean.columns)

In [ ]:
hadms = hf_icu_clean["hadm_id"].unique()

In [ ]:
d = {'subject_id':'first', 'hadm_id':'first', 'transfer_id':'first', 'eventtype':'first', 'careunit':'first', 'intime':'min', 'outtime':'max'}
for i in hadms:
    hadm = hf_icu_clean.loc[hf_icu_clean["hadm_id"] == i]
    consecutive_array = (hadm.department != hadm.department.shift()).cumsum().values
    new_hadm = hadm.groupby([consecutive_array, 'department']).agg(d).reset_index(level=1)
    newdf = pd.concat([newdf, new_hadm])
    
    

In [ ]:
hf_consecutive = newdf

In [ ]:
hadms = hf_consecutive["hadm_id"].unique()

In [ ]:
hf_consecutive = hf_consecutive.reset_index().drop("index", axis=1)

In [ ]:
hf_consecutive = hf_consecutive.reset_index()

In [ ]:
hf_consecutive

In [ ]:
icu_flag = 0
for i in hadms:
    hadm = hf_consecutive.loc[hf_consecutive["hadm_id"] == i]
    for index, row in hadm.iterrows():
        r = row
        if("ICU" in row["department"]):
            icu_flag = 1
        if((row["department"] != "Emergency Department") & (row["department"] != "Discharged") & ("ICU" not in row["department"])):
            if(icu_flag == 1):
                hf_consecutive.at[row["index"], "department"] = "Post-ICU " + row["department"] 
            else:
                hf_consecutive.at[row["index"], "department"] = "Pre-ICU " + row["department"] 
    icu_flag = 0

In [ ]:
hf_consecutive["transfer_duration"] = hf_consecutive["outtime"] - hf_consecutive["intime"]

In [ ]:
hadms = list(hf_consecutive["hadm_id"].unique())

In [ ]:
#consider only patients which have been in the ICU once
number_of_icu_visits = hf_consecutive.loc[hf_consecutive["department"].str.contains('Cardiac ICU|Medical ICU|Surgical ICU')].groupby("hadm_id").count().reset_index()

In [ ]:
to_remove = list(number_of_icu_visits.loc[number_of_icu_visits["subject_id"] > 1]["hadm_id"])

In [ ]:
hf_consecutive = hf_consecutive.loc[~hf_consecutive["hadm_id"].isin(to_remove)]

In [ ]:
hf_consecutive = hf_consecutive[['subject_id', 'hadm_id', 'transfer_id', 'eventtype', 'careunit', 'intime', 'outtime', 'department', 'transfer_duration']]

In [ ]:
to_merge = hf_icu_clean.drop(["Unnamed: 0", "department", "transfer_duration"], axis=1)

In [ ]:
final_df = hf_consecutive.merge(to_merge, how="left")

In [ ]:
hf_consecutive.to_csv("Kidney_Failure_Log.csv")